In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras 
import pandas as pd

In [2]:
orig_data = pd.read_csv("data_set/从11个特征预测学生的测试分数.csv")
X , Y = orig_data.iloc[:,0:10] , orig_data.iloc[:,-1] #切片
X = X.drop(columns=["school","classroom","student_id"],axis=1)#按列丢弃无关项
X = pd.get_dummies(X,columns=["school_setting","school_type","teaching_method","gender","lunch"],dtype=int)#独热编码 #orig_data.iloc[0:5,] X.iloc[0:5,]
X

,n_student,pretest,school_setting_Rural,school_setting_Suburban,school_setting_Urban,school_type_Non-public,school_type_Public,teaching_method_Experimental,teaching_method_Standard,gender_Female,gender_Male,lunch_Does not qualify,lunch_Qualifies for reduced/free lunch
0,20.0,62.0,0,0,1,1,0,0,1,1,0,1,0
1,20.0,66.0,0,0,1,1,0,0,1,1,0,1,0
2,20.0,64.0,0,0,1,1,0,0,1,0,1,1,0
3,20.0,61.0,0,0,1,1,0,0,1,1,0,1,0
4,20.0,64.0,0,0,1,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2128,30.0,39.0,0,0,1,0,1,0,1,1,0,1,0
2129,30.0,38.0,0,0,1,0,1,0,1,1,0,0,1
2130,30.0,45.0,0,0,1,0,1,0,1,1,0,0,1
2131,30.0,46.0,0,0,1,0,1,0,1,0,1,0,1


一层神经网络模型  
激活函数f(x)= x  
损失函数均方误差  
优化器Adam(默认比较优秀的优化器 结合了自适应梯度Adagrad和RMSProp防梯度急剧下降的优点)

In [3]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)#Sequential模型结构 多个网络层且无多余分支的堆叠 Dense就是常用的全连接层 运算就是output = activation(dot(input, kernel)+bias) units为该层的输出维度 而非输入维度  
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)) #adam（前期降低损失迅速，随迭代次数衰减学习率）学习率提高一点 
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) #整个数据一共遍历epoch遍 每一次拿batch_size个样本来算损失 所以一个epoch需要算epoch/batch_size遍 更新同样次数的参数
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 623us/step - loss: 1295.6588
Epoch 2/100
54/54 [==============================] - 0s 545us/step - loss: 80.8356
Epoch 3/100
54/54 [==============================] - 0s 508us/step - loss: 24.4455
Epoch 4/100
54/54 [==============================] - 0s 545us/step - loss: 23.6052
Epoch 5/100
54/54 [==============================] - 0s 508us/step - loss: 22.8562
Epoch 6/100
54/54 [==============================] - 0s 508us/step - loss: 22.0683
Epoch 7/100
54/54 [==============================] - 0s 527us/step - loss: 21.2628
Epoch 8/100
54/54 [==============================] - 0s 583us/step - loss: 20.5386
Epoch 9/100
54/54 [==============================] - 0s 545us/step - loss: 19.8143
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 19.1004 - val_loss: 22.5803
Epoch 11/100
54/54 [==============================] - 0s 639us/step - loss: 18.4475
Epoch 12/100
54/54 [==============================] - 0s 663us/st

换个优化器 随机梯度下降

In [4]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.SGD(learning_rate=0.01,clipvalue=0.5))#梯度裁剪 Gradient Clipping +-0.5
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 564us/step - loss: 1077.6292
Epoch 2/100
54/54 [==============================] - 0s 621us/step - loss: 152.2503
Epoch 3/100
54/54 [==============================] - 0s 583us/step - loss: 30.5355
Epoch 4/100
54/54 [==============================] - 0s 526us/step - loss: 23.3734
Epoch 5/100
54/54 [==============================] - 0s 527us/step - loss: 19.6744
Epoch 6/100
54/54 [==============================] - 0s 508us/step - loss: 18.1315
Epoch 7/100
54/54 [==============================] - 0s 527us/step - loss: 16.9032
Epoch 8/100
54/54 [==============================] - 0s 508us/step - loss: 15.8776
Epoch 9/100
54/54 [==============================] - 0s 526us/step - loss: 15.0975
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 14.5020 - val_loss: 13.5803
Epoch 11/100
54/54 [==============================] - 0s 564us/step - loss: 13.9997
Epoch 12/100
54/54 [==============================] - 0s 564us/s

自适应梯度 (对初始位置要求很严格 不然直接3000遍历还有1000损失 下降平缓均匀

In [5]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.Adagrad(learning_rate=0.01,epsilon=1e-6))
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 545us/step - loss: 8563.9258
Epoch 2/100
54/54 [==============================] - 0s 583us/step - loss: 7370.7891
Epoch 3/100
54/54 [==============================] - 0s 508us/step - loss: 6664.7520
Epoch 4/100
54/54 [==============================] - 0s 508us/step - loss: 6129.4712
Epoch 5/100
54/54 [==============================] - 0s 527us/step - loss: 5692.1816
Epoch 6/100
54/54 [==============================] - 0s 527us/step - loss: 5321.7017
Epoch 7/100
54/54 [==============================] - 0s 527us/step - loss: 4998.3179
Epoch 8/100
54/54 [==============================] - 0s 508us/step - loss: 4712.6245
Epoch 9/100
54/54 [==============================] - 0s 508us/step - loss: 4456.2007
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 4224.2217 - val_loss: 2799.4761
Epoch 11/100
54/54 [==============================] - 0s 526us/step - loss: 4012.4739
Epoch 12/100
54/54 [=======================

结合梯度平方的指数移动平均数来调节学习率的变化。能够在不稳定（Non-Stationary）的目标函数情况下进行很好地收敛。递归神经网络时的一个良好选择？

In [6]:
model = keras.Sequential(
    [
        keras.layers.Dense(units=1,activation='linear',name = 'IN_PUT',input_shape=(13,)),
     ]
)
model.compile(loss = keras.losses.MeanSquaredError(),optimizer = keras.optimizers.RMSprop(learning_rate=0.01, rho=0.9, epsilon=1e-6)) #rho衰减系数 epsilon数值稳定性的小常数？
model.fit(X,Y,epochs=100,validation_split=0.2,validation_freq=10,batch_size=32) 
model.summary() 
w1,b1 = model.get_layer("IN_PUT").get_weights()
print("特征对应相关的系数w1如下:")
for i in range(len(X.columns)):
    print("{:<40}  {:>20}".format(str(X.columns[i]),str(w1[i])))
print("系数b1:{}".format(str(b1)))

Epoch 1/100
54/54 [==============================] - 0s 564us/step - loss: 461.0370
Epoch 2/100
54/54 [==============================] - 0s 527us/step - loss: 21.8252
Epoch 3/100
54/54 [==============================] - 0s 489us/step - loss: 17.9568
Epoch 4/100
54/54 [==============================] - 0s 470us/step - loss: 15.7696
Epoch 5/100
54/54 [==============================] - 0s 677us/step - loss: 14.4441
Epoch 6/100
54/54 [==============================] - 0s 489us/step - loss: 13.5182
Epoch 7/100
54/54 [==============================] - 0s 489us/step - loss: 13.0097
Epoch 8/100
54/54 [==============================] - 0s 470us/step - loss: 12.6392
Epoch 9/100
54/54 [==============================] - 0s 489us/step - loss: 12.3692
Epoch 10/100
54/54 [==============================] - 0s 2ms/step - loss: 12.2267 - val_loss: 13.3907
Epoch 11/100
54/54 [==============================] - 0s 508us/step - loss: 12.1429
Epoch 12/100
54/54 [==============================] - 0s 489us/ste